In [1]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas pyarrow wandb plotly

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/d9/64/7fdfb9386511cd6805451e012c537073a79a958a58795c4e602e538c388c/opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/c9/e4/3e645a8f87577553194a2a15383a60d61b8381cf864e903e43e4c6eb58e0/mediapipe-0.10.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/4e/53/14405a47292b59235d811a2af8634aba188ccfd1a38ef4b8042f3447d79a/scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaini

In [2]:
import os
import math
import pandas as pd
import numpy as np
#Load parquet data into dataset_parquet for training.
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from glob import glob
from tqdm import tqdm
from collections import Counter
import random
import time
import wandb

2024-04-24 12:00:35.877024: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 12:00:36.091584: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 12:00:36.091640: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 12:00:36.092821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 12:00:36.187428: I tensorflow/core/platform/cpu_feature_g

In [3]:
class CONFIG:
    # root = os.path.join("/", "kaggle", "input", "asl-signs") 
    #root = os.path.join("..","data", "ASL-ds")
    root = os.path.join(".")
    DATA_LIMIT = 2000
    BATCH_SIZE = 16
    VIDEO_LENGTH = 36
    TRAIN_VAL_SPLIT = 0.80
    WANDB_RUN = "mediapipe-asl-dataset"


In [4]:
LIPS_IDXS0 = np.array([
        61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    ])

EYE_LEFT = np.array([33, 7, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 471, 470, 469, 472])
EYE_RIGHT = np.array([362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 476, 475, 474, 477])
# Landmark indices in original data
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)
LEFT_POSE_IDXS0 = np.array([502, 504, 506, 508, 510])
RIGHT_POSE_IDXS0 = np.array([503, 505, 507, 509, 511])

print("LIPS : ",len(LIPS_IDXS0))
print("EYE_LEFT : ",len(EYE_LEFT))
print("EYE_RIGHT : ",len(EYE_RIGHT))
print("LEFT_HAND : ",len(LEFT_HAND_IDXS0))
print("RIGHT_HAND : ",len(RIGHT_HAND_IDXS0))
print("LEFT_POSE : ",len(LEFT_POSE_IDXS0))
print("RIGHT_POSE : ",len(RIGHT_POSE_IDXS0))

all_selection = np.concatenate([LIPS_IDXS0, EYE_LEFT, EYE_RIGHT, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, LEFT_POSE_IDXS0, RIGHT_POSE_IDXS0])
print(len(all_selection))

LIPS :  40
EYE_LEFT :  20
EYE_RIGHT :  20
LEFT_HAND :  21
RIGHT_HAND :  21
LEFT_POSE :  5
RIGHT_POSE :  5
132


In [5]:
#this code sorts out a parquet files and rearrange the order to pose,face, left-hand, right-hand
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objs as go
import plotly.io as pio

ids = None

order_global = {"pose" : 10000, "face" : 1000, "left_hand" : 100, "right_hand" : 10}

def visualize_keypoints(frames : np.ndarray, point_size : int):
    if len(frames.shape) == 1:
        frames = np.array([frames])
    
    for frame in frames:
        frame = frame.reshape(-1, 3)
        sizes = point_size * np.ones(frame.shape[0])

        fig = go.Figure(data=go.Scatter(x=frame[:,0], y=2.5 - frame[:,1], mode='markers',
                                        marker=dict(
                                            size=sizes
                                            )))

    # Customize the layout
    fig.update_layout(title='visualization of human keypoints',
                        xaxis_title='',
                        yaxis_title='',
                        width=1000,
                        height=1600)

    fig.update_xaxes(range=[-0.2, 1.4])  # Set x-axis range from 0 to 6
    fig.update_yaxes(range=[0, 2.5])  # Set y-axis range from 10 to 20

    # Show the plot
    fig.show()


def process_parquet(ds, idxes = None):
    ret = []
    frames_unique = sorted(np.unique(ds["frame"]))
    for i,frame in enumerate(frames_unique):
        frame_ds = ds[ds['frame'] == frame]
        
        order = []
        for el in frame_ds["row_id"]:
            _frame, part, keypoint = el.split("-")
            order.append(order_global[part] - int(keypoint))

        order = np.array(order)
        frame_ds.iloc[:, 1] = order
        frame_ds = frame_ds.sort_values(by="row_id", ascending=False)
    
        vals = np.array(frame_ds[["x", "y", "z"]])
        if idxes is not None:
            vals = vals[idxes]
    
        vals = vals.flatten()

        ret.append(vals)
        
    return np.array(ret)


def process_parquet2(ds, idxes = None):
    ret = []    
    frame_size = 543
    it = len(ds) // frame_size
    assert it == len(ds) / frame_size
    
    for i in range(it):
        vals = ds.iloc[ i * frame_size : (i + 1 ) * frame_size ]        
        
        if idxes is not None:          
            vals = ds.iloc[idxes]
                        
        ret.append(np.array(vals[["x","y", "z"]]).flatten())
        
    return np.array(np.array(ret))

In [6]:
from sklearn.model_selection import train_test_split

#custom class to load data from Parquet files for training ML models.
class ParquetDatasetCached(keras.utils.Sequence):
    def __init__(self, dataset_folder, csv_file : str, batch_size=CONFIG.BATCH_SIZE, 
                 data_limit :int= CONFIG.DATA_LIMIT, check_if_file_exists = True, 
                 preprocessing_func=None, frame_length :int = CONFIG.VIDEO_LENGTH,
                 split : str = "train", train_val_split : float = CONFIG.TRAIN_VAL_SPLIT,
                 sort_by_counts : bool = True, **kwargs
                ):
        super().__init__(**kwargs)
        #taking keras sequence for .fit(), .evaluate(), .predict() methods
        #load csv - it has the path to parquet file, and another to store label
        self.csv_path = csv_file
        self.root_folder = dataset_folder
        self.batch_size = batch_size
        #optional pre-processing function to the parquet files.
        self.preprocessing_func = preprocessing_func
        
        self.csv_data = pd.read_csv(self.csv_path)
        
        self.all_files = []
        self.not_exists = []
        self.frame_length = frame_length

        
        for path, label in tqdm(list(zip(self.csv_data["path"], self.csv_data["sign"]))):
            prop_path = os.path.join(self.root_folder, path)
            
            if check_if_file_exists:
                if os.path.exists(prop_path):
                    self.all_files.append((prop_path, label))
                else:
                    self.not_exists.append(prop_path)
            else:
                self.all_files.append((prop_path, label))
                
                    
        self.all_files = np.array(self.all_files)
        self.unique_labels = np.unique(self.all_files[:, 1])
        self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
    
        # sort the values by popularity
        if sort_by_counts:
            cnt = Counter(self.all_files[:, 1])
            vals = []
            
            for i,row in enumerate(self.all_files):
                vals.append((int(1e6 * cnt[row[1]] + self.label_2_id [row[1]]),i))
            
            vals = np.array(sorted(vals)[::-1])
            self.all_files = self.all_files[vals[:,1]]

        
        if data_limit < 0:
            train_ds, val_ds = train_test_split(self.all_files, train_size=train_val_split, random_state=42)
        else:
            train_ds, val_ds = train_test_split(self.all_files[:data_limit], train_size=train_val_split, random_state=42)
            self.unique_labels = np.unique(self.all_files[:data_limit, 1])
            self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
            
        if split.lower() == "train":
            self.dataset = train_ds
            
        elif split.lower() == "val":
            self.dataset = val_ds 
            
        else:
            raise Exception("please specify split to be either train or val")
            
        self.cache_data()
                   

    def cache_data(self):
        self.cached_X, self.cached_Y = [], []
        
        pb = tqdm(range(len(self.dataset)), desc="Cacheing")

        for i in pb:
            
            path, label = self.dataset[i]
            df = pd.read_parquet(path)
                        
            one_hot_encoded_label = np.zeros(len(self.unique_labels))
            one_hot_encoded_label[self.label_2_id[label]] = 1

            self.cached_X.append(process_parquet2(df)) 
            self.cached_Y.append(one_hot_encoded_label)        

    def __len__(self):
        # Assuming each Parquet file should be one batch; adjust if necessary
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.preprocessing_func(self.cached_X[idx]), self.cached_Y[idx]                
        
    def on_epoch_end(self):
        # Shuffle files for the next epoch
        np.random.shuffle(self.dataset)


def preprocess(frames):
    current_length, num_features = frames.shape

    if current_length >= CONFIG.VIDEO_LENGTH:
            # TODO: a better than uniform value ? Could place gaussian in the middle
            random_start = random.randint(0, current_length - CONFIG.VIDEO_LENGTH)
            return np.nan_to_num(frames[random_start : (random_start + CONFIG.VIDEO_LENGTH)])
        
    # padd the video to contain zeros 
    return np.concatenate([np.nan_to_num(frames), np.zeros((CONFIG.VIDEO_LENGTH - current_length, num_features))], axis=0)

    
# Usage example
parquet_folder_path = CONFIG.root
train_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                check_if_file_exists = True,
                                split="train")

val_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                 check_if_file_exists= True,
                                 split="val")

print(f"cardinality of train : {len(train_dataset_parquet)}, cardinality of validation : {len(val_dataset_parquet)}")

Cacheing: 100%|██████████| 400/400 [00:05<00:00, 74.82it/s]

cardinality of train : 1600, cardinality of validation : 400


In [7]:
X_shape = train_dataset_parquet[0][0].shape
Y_shape = train_dataset_parquet[0][1].shape
print(f"X_shape = {X_shape}, Y_shape = {Y_shape}")

X_shape = (36, 1629), Y_shape = (5,)


In [8]:
print(train_dataset_parquet.label_2_id)

{'donkey': 0, 'listen': 1, 'look': 2, 'mouse': 3, 'shhh': 4}


In [9]:
def dataset_train_generator():
    # Instantiate your existing dataset loader

    for i in range(len(train_dataset_parquet)):
        X_batch, Y_batch = train_dataset_parquet[i]
        yield X_batch, Y_batch
        
def dataset_val_generator():
    # Instantiate your existing dataset loader

    for i in range(len(val_dataset_parquet)):
        X_batch, Y_batch = val_dataset_parquet[i]
        yield X_batch, Y_batch        


train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_train_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_val_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

2024-04-24 12:01:06.608179: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22462 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
start = time.time()
isnans =False

f = True
labels_batches = []
for el in tqdm(train_dataset):
    if f:
        print(el[0].shape, el[1].shape)
        f = False
    labels_batches.append(el[1])
        
    isnans |= np.any(np.isnan(el[0]))
    if isnans:
        print("FOUND NAN!")
        break


print(f"Iterating through dataset took : {round( time.time() - start , 4)}s")

29it [00:00, 158.56it/s]

(16, 36, 1629) (16, 5)


100it [00:00, 201.31it/s]

Iterating through dataset took : 0.5044s


# Train experiment

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from wandb.keras import WandbMetricsLogger
from keras.callbacks import Callback
import tensorflow as tf
import wandb


class CosineAnnealingLearningRateScheduler(Callback):
    def __init__(self, max_lr, min_lr, T_max):
        super(CosineAnnealingLearningRateScheduler, self).__init__()
        self.max_lr = max_lr  # Maximum learning rate (i.e., start learning rate)
        self.min_lr = min_lr  # Minimum learning rate
        self.T_max = T_max    # Specifies the number of epochs per cycle
        self.t = 0            # Current epoch

    def on_epoch_begin(self, epoch, logs=None):
        self.t += 1
        cos = np.cos(np.pi * (self.t % self.T_max) / self.T_max)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + cos)

        keras.backend.set_value(self.model.optimizer.lr, lr)

def keras_train(model, filepath : str, X_shape, max_lr = 1e-4, min_lr = 2.5e-5, T_max=101, epochs=200, run_name="",
                mediapipe_features = "all", USE_WANDB=True,
               verbose="auto"): 
    
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                 monitor="val_categorical_accuracy",
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode="max",
                                                 save_freq="epoch")
    
    cosine_annealer = CosineAnnealingLearningRateScheduler(max_lr=max_lr,
                                                           min_lr=min_lr,
                                                           T_max=T_max)
    
    #Adam Optimizer - fixed learning rate.
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=max_lr, clipnorm=1.)

    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.build(X_shape)
    
    
    callbacks  = [checkpoint, cosine_annealer]
    
    if USE_WANDB:
        wandb.init(project=CONFIG.WANDB_RUN,
                        name=run_name,
                        notes="Model summary : \n" + str(model.summary()),
                        config={"max_lr" : max_lr, 
                                "min_lr" : 5e-5, 
                                "scheduler" : "cosineAnnealer", 
                                "epochs" : epochs, 
                                "T_max" : T_max, 
                                "train_size" : len(train_dataset_parquet.dataset),
                                "val_size" : len(val_dataset_parquet.dataset),
                                "unique_classes" : len(train_dataset_parquet.unique_labels), 
                                "video_length" : CONFIG.VIDEO_LENGTH,
                                "features" : mediapipe_features
                                })
        callbacks.append(WandbMetricsLogger())


    
    history = model.fit(train_dataset, epochs=epochs, validation_data = val_dataset, callbacks=callbacks, verbose=verbose)
    
    if USE_WANDB:      
        wandb.finish()
    
    return history

## LSTM

(36, 1629)

In [35]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-LSTM16-LR-Dense256-fs=15-cls=3.tf"),
#            run_name="AK-LSTM16-LR-Dense256-fs=15-cls=3", X_shape=X_shape,
#            USE_WANDB=True)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 16)                105344    
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 16)                0         
                                                                 
 dense_50 (Dense)            (None, 256)               4352      
                                                                 
 dense_51 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 110981 (433.52 KB)
Trainable params: 110981 (433.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


model.summary()
"""
keras_train(model, filepath=os.path.join("models", "AK-LSTM32-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM32-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)
"""

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 32)                212736    
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 32)                0         
                                                                 
 dense_52 (Dense)            (None, 256)               8448      
                                                                 
 dense_53 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 222469 (869.02 KB)
Trainable params: 222469 (869.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


'\nkeras_train(model, filepath=os.path.join("models", "AK-LSTM32-LR-Dense256-fs=15-cls=3.tf"),\n            run_name="AK-LSTM32-LR-Dense256-fs=15-cls=3",\n            USE_WANDB=True)\n'

In [37]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
"""
keras_train(model, filepath=os.path.join("models", "AK-LSTM64-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM64-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)
"""

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 64)                433664    
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 64)                0         
                                                                 
 dense_54 (Dense)            (None, 256)               16640     
                                                                 
 dense_55 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 451589 (1.72 MB)
Trainable params: 451589 (1.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


'\nkeras_train(model, filepath=os.path.join("models", "AK-LSTM64-LR-Dense256-fs=15-cls=3.tf"),\n            run_name="AK-LSTM64-LR-Dense256-fs=15-cls=3",\n            USE_WANDB=True)\n'

In [68]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
"""
keras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM128-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)
"""

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 128)               900096    
                                                                 
 leaky_re_lu_39 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_126 (Dense)           (None, 256)               33024     
                                                                 
 dense_127 (Dense)           (None, 5)                 1285      
                                                                 
Total params: 934405 (3.56 MB)
Trainable params: 934405 (3.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


'\nkeras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=15-cls=3.tf"),\n            run_name="AK-LSTM128-LR-Dense256-fs=15-cls=3",\n            USE_WANDB=True)\n'

In [70]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(512, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
"""
keras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM128-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)
"""

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 512)               4386816   
                                                                 
 leaky_re_lu_41 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_130 (Dense)           (None, 256)               131328    
                                                                 
 dense_131 (Dense)           (None, 5)                 1285      
                                                                 
Total params: 4519429 (17.24 MB)
Trainable params: 4519429 (17.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


'\nkeras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=15-cls=3.tf"),\n            run_name="AK-LSTM128-LR-Dense256-fs=15-cls=3",\n            USE_WANDB=True)\n'

In [40]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(512, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
#keras_train(model, filepath=os.path.join("models", "AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf"),
#            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
#            USE_WANDB=True)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 36, 512)           4386816   
                                                                 
 flatten_11 (Flatten)        (None, 18432)             0         
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 18432)             0         
                                                                 
 dense_60 (Dense)            (None, 256)               4718848   
                                                                 
 dense_61 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 9106949 (34.74 MB)
Trainable params: 9106949 (34.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## GRU

In [41]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
#keras_train(model, filepath=os.path.join("models", "AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf"),
#            run_name="AK-GRU16-LR-Dense256-gelu-fs=15-cls=3",
#            USE_WANDB=True)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                79056     
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 16)                0         
                                                                 
 dense_62 (Dense)            (None, 256)               4352      
                                                                 
 dense_63 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 84693 (330.83 KB)
Trainable params: 84693 (330.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()
#keras_train(model, filepath=os.path.join("models", "AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf"),
#            run_name="AK-GRU32-LR-Dense256-gelu-fs=15-cls=3",
#            USE_WANDB=True)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 32)                159648    
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 32)                0         
                                                                 
 dense_64 (Dense)            (None, 256)               8448      
                                                                 
 dense_65 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 169381 (661.64 KB)
Trainable params: 169381 (661.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-GRU64-LR-Dense256-gelu-cls=3.tf"),
#            run_name="AK-GRU64-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 64)                325440    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 64)                0         
                                                                 
 dense_66 (Dense)            (None, 256)               16640     
                                                                 
 dense_67 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 343365 (1.31 MB)
Trainable params: 343365 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-GRU128-LR-Dense256-gelu-cls=3.tf"),
#            run_name="AK-GRU128-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 128)               675456    
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_68 (Dense)            (None, 256)               33024     
                                                                 
 dense_69 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 709765 (2.71 MB)
Trainable params: 709765 (2.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(256, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-GRU256-LR-Dense256-gelu-cls=3.tf"),
#            run_name="AK-GRU256-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 256)               1449216   
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_70 (Dense)            (None, 256)               65792     
                                                                 
 dense_71 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 1516293 (5.78 MB)
Trainable params: 1516293 (5.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(512, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-GRU512-LR-Dense256-gelu-cls=3.tf"),
#            run_name="AK-GRU512-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_5 (GRU)                 (None, 512)               3291648   
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_72 (Dense)            (None, 256)               131328    
                                                                 
 dense_73 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3424261 (13.06 MB)
Trainable params: 3424261 (13.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Bidirectional LSTM

In [51]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, X_shape=(None, X_shape[0], X_shape[1]), filepath=os.path.join("models", "AK-BLSTM16-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BLSTM16-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_14 (Bidirect  (None, 36, 32)            210688    
 ional)                                                          
                                                                 
 flatten_16 (Flatten)        (None, 1152)              0         
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 1152)              0         
                                                                 
 dense_82 (Dense)            (None, 256)               295168    
                                                                 
 dense_83 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 507141 (1.93 MB)
Trainable params: 507141 (1.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [52]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, X_shape=(None, X_shape[0], X_shape[1]), filepath=os.path.join("models", "AK-BLSTM32-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BLSTM32-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_15 (Bidirect  (None, 36, 64)            425472    
 ional)                                                          
                                                                 
 flatten_17 (Flatten)        (None, 2304)              0         
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 2304)              0         
                                                                 
 dense_84 (Dense)            (None, 256)               590080    
                                                                 
 dense_85 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 1016837 (3.88 MB)
Trainable params: 1016837 (3.88 MB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [53]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model,X_shape=(None, X_shape[0], X_shape[1]), filepath=os.path.join("models", "AK-BLSTM64-LR-Dense256-gelu-cls=3.tf"),
#            run_name="AK-BLSTM64-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_16 (Bidirect  (None, 36, 128)           867328    
 ional)                                                          
                                                                 
 flatten_18 (Flatten)        (None, 4608)              0         
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 4608)              0         
                                                                 
 dense_86 (Dense)            (None, 256)               1179904   
                                                                 
 dense_87 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 2048517 (7.81 MB)
Trainable params: 2048517 (7.81 MB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [54]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model,X_shape=(None, X_shape[0], X_shape[1]), filepath=os.path.join("models", "AK-BLSTM128-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BLSTM128-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_17 (Bidirect  (None, 36, 256)           1800192   
 ional)                                                          
                                                                 
 flatten_19 (Flatten)        (None, 9216)              0         
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 9216)              0         
                                                                 
 dense_88 (Dense)            (None, 256)               2359552   
                                                                 
 dense_89 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 4161029 (15.87 MB)
Trainable params: 4161029 (15.87 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

In [55]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model,X_shape=(None, X_shape[0], X_shape[1]), filepath=os.path.join("models", "AK-BLSTM256-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BLSTM256-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_18 (Bidirect  (None, 36, 512)           3862528   
 ional)                                                          
                                                                 
 flatten_20 (Flatten)        (None, 18432)             0         
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 18432)             0         
                                                                 
 dense_90 (Dense)            (None, 256)               4718848   
                                                                 
 dense_91 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 8582661 (32.74 MB)
Trainable params: 8582661 (32.74 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

## Bidirectional GRU

In [56]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-BGRU16-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BGRU16-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_19 (Bidirect  (None, 36, 32)            158112    
 ional)                                                          
                                                                 
 flatten_21 (Flatten)        (None, 1152)              0         
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 1152)              0         
                                                                 
 dense_92 (Dense)            (None, 256)               295168    
                                                                 
 dense_93 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 454565 (1.73 MB)
Trainable params: 454565 (1.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [57]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-BGRU32-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BGRU32-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_20 (Bidirect  (None, 36, 64)            319296    
 ional)                                                          
                                                                 
 flatten_22 (Flatten)        (None, 2304)              0         
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 2304)              0         
                                                                 
 dense_94 (Dense)            (None, 256)               590080    
                                                                 
 dense_95 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 910661 (3.47 MB)
Trainable params: 910661 (3.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [58]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-BGRU64-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BGRU64-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_21 (Bidirect  (None, 36, 128)           650880    
 ional)                                                          
                                                                 
 flatten_23 (Flatten)        (None, 4608)              0         
                                                                 
 leaky_re_lu_34 (LeakyReLU)  (None, 4608)              0         
                                                                 
 dense_96 (Dense)            (None, 256)               1179904   
                                                                 
 dense_97 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 1832069 (6.99 MB)
Trainable params: 1832069 (6.99 MB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [59]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-BGRU128-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BGRU128-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_22 (Bidirect  (None, 36, 256)           1350912   
 ional)                                                          
                                                                 
 flatten_24 (Flatten)        (None, 9216)              0         
                                                                 
 leaky_re_lu_35 (LeakyReLU)  (None, 9216)              0         
                                                                 
 dense_98 (Dense)            (None, 256)               2359552   
                                                                 
 dense_99 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 3711749 (14.16 MB)
Trainable params: 3711749 (14.16 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

In [60]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))

model.build((None, X_shape[0], X_shape[1]))
model.summary()

#keras_train(model, filepath=os.path.join("models", "AK-BGRU256-LR-Dense256-gelu-cls=3"),
#            run_name="AK-BGRU256-LR-Dense256-gelu-cls=3",
#            USE_WANDB=True)

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_23 (Bidirect  (None, 36, 512)           2898432   
 ional)                                                          
                                                                 
 flatten_25 (Flatten)        (None, 18432)             0         
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 18432)             0         
                                                                 
 dense_100 (Dense)           (None, 256)               4718848   
                                                                 
 dense_101 (Dense)           (None, 5)                 1285      
                                                                 
Total params: 7618565 (29.06 MB)
Trainable params: 7618565 (29.06 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

# Transformer

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [61]:
embed_dim = 64  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(CONFIG.VIDEO_LENGTH, 1629))
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)


model.summary()

#keras_train(model, filepath=os.path.join("models", f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128.tf"),
#            run_name=f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128",
#                max_lr=1e-4,
#            min_lr=2.5e-5,
#            X_shape=(None, X_shape[0], X_shape[1]),
#            T_max=101,
#            epochs=200,
#            mediapipe_features="reduced-LipsEyesHandsPose-scaled",
#            USE_WANDB=True)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng_7 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_7 (Trans  (None, 36, 1629)          21665828  
 formerBlock)                                                    
                                                                 
 global_average_pooling1d_7  (None, 1629)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_30 (Dropout)        (None, 1629)              0   

In [62]:
embed_dim = 64  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(CONFIG.VIDEO_LENGTH, 1629))
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()
#keras_train(model, filepath=os.path.join("models", f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128.tf"),
#            run_name=f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128",
#                max_lr=1e-4,
#            min_lr=2.5e-5,
#            X_shape=(None, X_shape[0], X_shape[1]),
#            T_max=101,
#            epochs=200,
#            mediapipe_features="reduced-LipsEyesHandsPose-scaled",
#            USE_WANDB=True)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng_8 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_8 (Trans  (None, 36, 1629)          21457252  
 formerBlock)                                                    
                                                                 
 global_average_pooling1d_8  (None, 1629)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_34 (Dropout)        (None, 1629)              0   

In [63]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(CONFIG.VIDEO_LENGTH, 1629))
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()
#keras_train(model, filepath=os.path.join("models", f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128.tf"),
#            run_name=f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128",
#                max_lr=1e-4,
#            min_lr=2.5e-5,
#            X_shape=(None, X_shape[0], X_shape[1]),
#            T_max=101,
#            epochs=200,
#            mediapipe_features="all",
#            USE_WANDB=True)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng_9 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_9 (Trans  (None, 36, 1629)          21665828  
 formerBlock)                                                    
                                                                 
 global_average_pooling1d_9  (None, 1629)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_38 (Dropout)        (None, 1629)              0   

In [64]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(CONFIG.VIDEO_LENGTH, 1629))
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()
#keras_train(model, filepath=os.path.join("models", f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128.tf"),
#            run_name=f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128",
#                max_lr=1e-4,
#            min_lr=2.5e-5,
#            X_shape=(None, X_shape[0], X_shape[1]),
#            T_max=101,
#            epochs=200,
#            mediapipe_features="all",
#            USE_WANDB=True)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng_10 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_10 (Tran  (None, 36, 1629)          21457252  
 sformerBlock)                                                   
                                                                 
 global_average_pooling1d_1  (None, 1629)              0         
 0 (GlobalAveragePooling1D)                                      
                                                                 
 dropout_42 (Dropout)        (None, 1629)              0  

In [65]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 96  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(CONFIG.VIDEO_LENGTH, 1629))
embedding_layer = TokenAndPositionEmbedding(X_shape[0], X_shape[1])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(X_shape[1], num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = Dense(len(train_dataset_parquet.unique_labels), activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()
#keras_train(model, filepath=os.path.join("models", f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128.tf"),
#            run_name=f"AK-Transformer-e={embed_dim}-h={num_heads}-ff={ff_dim}-D=0.1-Dense=128",
#                max_lr=1e-4,
#            min_lr=2.5e-5,
#            X_shape=(None, X_shape[0], X_shape[1]),
#            T_max=101,
#            epochs=200,
#            mediapipe_features="all",
#            USE_WANDB=True)

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 36, 1629)]        0         
                                                                 
 token_and_position_embeddi  (None, 36, 1629)          58644     
 ng_11 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_11 (Tran  (None, 36, 1629)          42800442  
 sformerBlock)                                                   
                                                                 
 global_average_pooling1d_1  (None, 1629)              0         
 1 (GlobalAveragePooling1D)                                      
                                                                 
 dropout_46 (Dropout)        (None, 1629)              0  